## Streaming

<img src="./assets/LC_streaming.png" width="500">

Streaming reduces the latency between generating data and user receiving them

In [1]:
from langchain.agents import create_agent

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    system_prompt="You are a full-stack comedian.",
)

### No Streaming - Invoke

No streaming - everything once completed

In [3]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content": "Tell me a joke."}
    ]}
)

print(result["messages"][-1].content)

Why don’t skeletons fight each other?

They don’t have the guts!


### Values

Stream message by message - Eg: First human message. After that AI message...

In [5]:
for step in agent.stream(
    {"messages":[
        {"role": "user", "content": "Tell me a dad joke"}
    ]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a dad joke
================================== Ai Message ==================================

Why don’t skeletons fight each other?  

They don’t have the guts. 👻😂


### Messages

Messages stream data token by token

In [8]:
for token, metadata in agent.stream(
    {"messages":[
        {"role": "user", "content": "Write a family friendly poem"}
    ]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

**A Picnic Under the Sun**

We gather round a blanket, warm and soft,  
With sandwiches, lemonade, and a loaf of love.  
The kids giggle as the kites start to rise,  
Above their heads, a gentle summer sky.

A robin sings the anthem of bright new days,  
While squirrels trade acorns in the trees' bright blaze.  
We share stories, jokes, and a slice of pie,  
Under the canopy of clouds drifting by.

Evening whispers, stars peek to greet,  
We fold the day, tucked beneath the warm, blue seat.  
With hearts full of joy, we dream of tomorrow—  
A family’s simple, sweet, bright, sunlit sorrow.

### Streaming in Tools

We can deliver information to user even before the final result is ready via streaming. In tools, we can provide status information to the user

In [19]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer
from langchain_core.tools import tool

@tool
def get_weather(city: str) -> str:
    """Get weather information for a given city."""
    writer = get_stream_writer()

    writer(f"Fetching weather information for {city}...\n")
    writer(f"Fetched data from weather API for {city}...\n")

    return f"The weather in {city} is sunny with a high of 25°C."

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    system_prompt="You are a helpful assistant that provides weather information.",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages":[
        {"role": "user", "content": "What's the weather like in San Francisco?"}
    ]},
    stream_mode=["values","custom"],
):
    print(chunk) 

('values', {'messages': [HumanMessage(content="What's the weather like in San Francisco?", additional_kwargs={}, response_metadata={}, id='09f8e523-f3d0-4372-915b-3fe65612a519')]})
('values', {'messages': [HumanMessage(content="What's the weather like in San Francisco?", additional_kwargs={}, response_metadata={}, id='09f8e523-f3d0-4372-915b-3fe65612a519'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b-cloud', 'created_at': '2025-10-31T06:23:19.184269294Z', 'done': True, 'done_reason': 'stop', 'total_duration': 489759739, 'load_duration': None, 'prompt_eval_count': 128, 'prompt_eval_duration': None, 'eval_count': 40, 'eval_duration': None, 'model_name': 'gpt-oss:20b-cloud', 'model_provider': 'ollama'}, id='lc_run--2776bccf-1e2e-46fc-916f-e2fe68464745-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '4680a903-3ed6-48d0-933b-0bc5ccdad740', 'type': 'tool_call'}], usage_metadata={'input_tokens': 128, 'output_tokens': 40

In [25]:
for chunk in agent.stream(
    {"messages":[
        {"role": "user", "content": "What's the weather like in SF?"}
    ]},
    stream_mode=["custom"],
):
    print(f"{chunk}")

('custom', 'Fetching weather information for SF...\n')
('custom', 'Fetched data from weather API for SF...\n')


In [20]:
for chunk in agent.stream(
    {"messages":[
        {"role": "user", "content": "What's the weather like in SF?"}
    ]},
    stream_mode=["values","custom"],
):
    if chunk[0] == "custom":
        print(f"{chunk[1]}", end="")

Fetching weather information for San Francisco...
Fetched data from weather API for San Francisco...
